Imports

In [0]:
import requests
from helpers import RAW, table_exists, CATALOG, BUCKET_BASE, TAG, _root, CREDS_NAME
from pyspark.sql import SparkSession

Criação do CATALOGO, SCHEMAS e VOLUMES

In [0]:
spark = SparkSession.getActiveSession()

# 1. root
ddl_root = f"""
CREATE EXTERNAL LOCATION IF NOT EXISTS {CATALOG}_root
  URL '{BUCKET_BASE}'
  WITH (STORAGE CREDENTIAL `{CREDS_NAME}`);
"""

# 2. Catálogo e SCHEMAs
ddl_catalog = f"""
CREATE CATALOG IF NOT EXISTS {CATALOG}
  MANAGED LOCATION '{BUCKET_BASE}/uc_root';
"""

layers = ["raw", "bronze", "quarentine", "silver", "gold"]
ddl_schemas = "\n".join(
    f"""
    CREATE SCHEMA IF NOT EXISTS {CATALOG}.{layer}
      MANAGED LOCATION '{BUCKET_BASE}/{layer}/_tables/';
    """.strip()
    for layer in layers
)

#3. Volumes UC (RAW) + External Volumes
ddl_volumes = f"""
-- RAW ── Volume UC (DBFS)
CREATE VOLUME IF NOT EXISTS {CATALOG}.raw.{TAG}_ingest
COMMENT 'Landing zone RAW {TAG.upper()}';

""" + "\n".join(
    f"""
    -- {layer.upper()} ── external volume
    CREATE EXTERNAL VOLUME IF NOT EXISTS {CATALOG}.{layer}.{TAG}_{layer}
    LOCATION '{_root(layer, "volumes")}'
    COMMENT 'Camada {layer} – dados {TAG.upper()}';
    """.strip()
    for layer in ["bronze", "quarentine", "silver", "gold"]
)

for stmt in [ddl_root, ddl_catalog, ddl_schemas, ddl_volumes]:
    for sql in filter(None, (s.strip() for s in stmt.split(";"))):
        spark.sql(sql)         


Execution

In [0]:
for tt in RAW.TAXI_TYPES:
    for mm in RAW.MONTHS:
        fname = f"{tt}_tripdata_{RAW.YEAR}-{mm}.parquet"
        url   = f"{RAW.BASE_URL}/{fname}"
        dst   = (f"{RAW.DEST_ROOT}/"
                 f"taxi_type={tt}/year={RAW.YEAR}/month={mm}/{fname}")

        with tempfile.NamedTemporaryFile(delete=False) as tmp:
            with requests.get(url, stream=True, timeout=180) as r:
                r.raise_for_status()
                for blk in r.iter_content(RAW.CHUNK):
                    tmp.write(blk)
            local_tmp = tmp.name

        dbutils.fs.cp("file:" + local_tmp, dst)

        os.remove(local_tmp)
        print("Arquivo salvo em:", dst)

print("Parquets disponíveis no volume/s3")